In [1]:
# https://blog.csdn.net/marsjhao/article/details/68490105 https://blog.csdn.net/u014281392/article/details/77103747
import numpy as np
import scipy.io as sio
np.random.seed(355)  # for reproducibility
from keras.models import Sequential 
from keras import layers
from keras.layers import Dropout, Bidirectional,Activation,Flatten,GRU,LSTM
from keras.layers import Dense, Input, TimeDistributed,RepeatVector
from keras.utils import np_utils
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras import Model
from keras.optimizers import Adam
#from keras.utils import multi_gpu_model
import math
import time
import config_weibo_1 as cf
import six.moves.cPickle as pickle
from keras_self_attention import SeqSelfAttention
import keras

Using TensorFlow backend.
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

observation time 3599
the number of time interval: 6
time interval: 600


In [2]:
batch_size = 32
epochs = 20
CELL_SIZE=32
OUTPUT_SIZE=20
TIME_STEPS = cf.n_time_interval
datasetName =cf.datasetName
INPUT_SIZE = len(cf.degree_interval_list)+1
input_shape = (TIME_STEPS, INPUT_SIZE)
PRE_INPUT_SIZE = 6
PRE_OUTPUT_SIZE = 12
print (input_shape)

(6, 10)


In [3]:
def Z_Score(data):
    lenth = len(data)
    total = sum(data)
    ave = float(total)/lenth
    tempsum = sum([pow(data[i] - ave,2) for i in range(lenth)])
    tempsum = pow(float(tempsum)/lenth,0.5)
    for i in range(lenth):
        data[i] = (data[i] - ave)/tempsum
    return data

In [4]:
id_train, x_train,pre_x_train, L, y_train, sz_train, time_train, vocabulary_size = pickle.load(open(cf.train_pkl, 'rb'))
id_test, x_test,pre_x_test, L_test, y_test, sz_test, time_test, _ = pickle.load(open(cf.test_pkl, 'rb'))
id_val, x_val,pre_x_val, L_val, y_val, sz_val, time_val, _ = pickle.load(open(cf.val_pkl, 'rb'))
for i in range(len(x_train)):
    x_train[i] = np.array(x_train[i])
for i in range(len(x_val)):
    x_val[i] = np.array(x_val[i])
for i in range(len(x_test)):
    x_test[i] = np.array(x_test[i])
x_val = np.array(x_val)
pre_x_val = np.array(pre_x_val)
x_train = np.array(x_train)
pre_x_train = np.array(pre_x_train)
x_test = np.array(x_test)
pre_x_test = np.array(pre_x_test)

y_val = np.array(y_val)
y_train = np.array(y_train)
y_test = np.array(y_test)
print(x_train.shape)
print(pre_x_train.shape)
print(x_test.shape)
print(pre_x_test.shape)
print(x_val.shape)
print(pre_x_val.shape)
print(y_train.shape)
# x_train = x_train.reshape(x_train.shape[0], TIME_STEPS, INPUT_SIZE, 1)
# x_test = x_test.reshape(x_test.shape[0], TIME_STEPS, INPUT_SIZE, 1)
# x_val = x_val.reshape(x_val.shape[0], TIME_STEPS, INPUT_SIZE, 1)

(29529, 6, 10)
(29529, 6, 6)
(6327, 6, 10)
(6327, 6, 6)
(6327, 6, 10)
(6327, 6, 6)
(29529,)


In [5]:
for i in range(6):
    for j in range(6):
        Z_Score(pre_x_train[:,i,j])
        Z_Score(pre_x_test[:,i,j])
        Z_Score(pre_x_val[:,i,j])

In [6]:
## pre autoencoder

retrunSeq=True
input_data = Input((None,INPUT_SIZE))
encoder=GRU(units=OUTPUT_SIZE*2,  activation='tanh',return_sequences=retrunSeq, name="gru1")(input_data)
encoder=GRU(units=OUTPUT_SIZE,  activation='tanh',return_sequences=retrunSeq, name="gru2")(encoder)
encoder_out=Dense(INPUT_SIZE,activation='tanh')(encoder)
encoder_model = Model(inputs=input_data, outputs=encoder_out)

decoder=GRU(units=OUTPUT_SIZE, activation='tanh',return_sequences=retrunSeq,name="de_gru1")(encoder_out)
decoder=GRU(units=OUTPUT_SIZE*2, activation='tanh', return_sequences=retrunSeq, name="de_gru2")(decoder)
decoder_out=Dense(PRE_INPUT_SIZE,activation='relu')(decoder)
autoencoder=Model(input_data,decoder_out)
 
#Compile
#autoencoder = multi_gpu_model(autoencoder, 1)
autoencoder.compile(loss='mean_squared_error',optimizer='adadelta',metrics=['mse'])
autoencoder.summary()  #Print

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', patience = 3,verbose = 1, factor=0.5, min_lr = 0.000001)
#Train
autoencoder.fit(x_train, pre_x_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val,pre_x_val), 
          shuffle=True, verbose=1,callbacks=[learning_rate_reduction])

#encoderModel Migration

x_train_mid = encoder_model.predict(x_train)
x_test_mid = encoder_model.predict(x_test)
x_val_mid = encoder_model.predict(x_val)

#x_train = np.concatenate((x_train,x_train_plus),axis=2)
#x_test =np.concatenate((x_test,x_test_plus),axis=2)
#x_val = np.concatenate((x_val,x_val_plus),axis=2)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 10)          0         
_________________________________________________________________
gru1 (GRU)                   (None, None, 40)          6120      
_________________________________________________________________
gru2 (GRU)                   (None, None, 20)          3660      
_________________________________________________________________
dense_1 (Dense)              (None, None, 10)          210       
_________________________________________________________________
de_gru1 (GRU)                (None, None, 20)          1860      
_________________________________________________________________
de_gru2 (GRU)                (None, None, 40)          7320      
_________________________________________________________________
dense_2 (Dense)              (None, None, 6)           246 

D:\anaconda\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_mse,loss,mse,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


29529/29529 [==============================] - 26s 877us/step - loss: 0.5380 - mse: 0.5380 - val_loss: 0.5063 - val_mse: 0.5063
Epoch 3/20
29529/29529 [==============================] - 27s 921us/step - loss: 0.4819 - mse: 0.4819 - val_loss: 0.4768 - val_mse: 0.4768
Epoch 4/20
29529/29529 [==============================] - 28s 955us/step - loss: 0.4549 - mse: 0.4549 - val_loss: 0.4409 - val_mse: 0.4409
Epoch 5/20
29529/29529 [==============================] - 27s 923us/step - loss: 0.4337 - mse: 0.4337 - val_loss: 0.4212 - val_mse: 0.4212
Epoch 6/20
29529/29529 [==============================] - 29s 989us/step - loss: 0.4193 - mse: 0.4193 - val_loss: 0.4168 - val_mse: 0.4168
Epoch 7/20
29529/29529 [==============================] - 28s 934us/step - loss: 0.4073 - mse: 0.4073 - val_loss: 0.4106 - val_mse: 0.4106
Epoch 8/20
29529/29529 [==============================] - 36s 1ms/step - loss: 0.4005 - mse: 0.4005 - val_loss: 0.3843 - val_mse: 0.3843
Epoch 9/20
29529/29529 [================

In [7]:
## LSTM 
model = Sequential()
model.add(GRU(units = OUTPUT_SIZE, activation='tanh', input_shape=input_shape,return_sequences=True))
model.add(GRU(units = OUTPUT_SIZE, activation='tanh',return_sequences=True))
model.add(Flatten()) #Pull into one-dimensional data
model.add(Dense(16, activation='relu')) 
model.add(Dense(1, activation='relu')) #Fully connected layer
# model.add(Dense(1, activation='relu')) #Fully connected layer

#Compile
#model = multi_gpu_model(model, 1)
model.compile(loss='mean_squared_error',optimizer='adadelta',metrics=['mse'])
model.summary()  #Print

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', patience = 3,verbose = 1, factor=0.5, min_lr = 0.000000001)
#Train
f = open(datasetName+'result_epoch_rnn.csv','w')
for epoch in range(epochs):
    model.fit(x_train_mid, y_train, batch_size=batch_size, nb_epoch=1, validation_data=(x_val_mid, y_val), 
              shuffle=True, verbose=1,callbacks=[learning_rate_reduction])
    pre_test_score = model.evaluate(x_test_mid, y_test, verbose=0)
    pre_val_score  = model.evaluate(x_val_mid, y_val, verbose=0)
    f.write(str(epoch)+',Pre Test score,'+ str(pre_test_score[0])+',Pre Val_score,'+str(pre_val_score[0])+'\n')
    print('Pre Test score:', pre_test_score[0])
f.close()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 6, 20)             1860      
_________________________________________________________________
gru_2 (GRU)                  (None, 6, 20)             2460      
_________________________________________________________________
flatten_1 (Flatten)          (None, 120)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                1936      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 6,273
Trainable params: 6,273
Non-trainable params: 0
_________________________________________________________________


D:\anaconda\lib\site-packages\ipykernel_launcher.py:20: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 16s 536us/step - loss: 3.0562 - mse: 3.0562 - val_loss: 2.4888 - val_mse: 2.4888  - ETA: 1s - loss: 3.1


D:\anaconda\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_mse,loss,mse,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Pre Test score: 2.3851101409396005
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
  288/29529 [..............................] - ETA: 12s - loss: 2.4449 - mse: 2.44

D:\anaconda\lib\site-packages\ipykernel_launcher.py:20: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


29529/29529 [==============================] - 14s 481us/step - loss: 2.2554 - mse: 2.2554 - val_loss: 2.3373 - val_mse: 2.3373
Pre Test score: 2.192839645031826
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 17s 568us/step - loss: 2.2039 - mse: 2.2039 - val_loss: 2.4668 - val_mse: 2.4668
Pre Test score: 2.2975569067548496
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 15s 511us/step - loss: 2.1799 - mse: 2.1799 - val_loss: 2.2841 - val_mse: 2.2841
Pre Test score: 2.1506509771463107
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 14s 462us/step - loss: 2.1622 - mse: 2.1622 - val_loss: 2.3434 - val_mse: 2.3434
Pre Test score: 2.1911669859336755
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 14s 472us/step - loss: 2.1494 - mse: 2.1494 - val_loss: 2.2548 - v

In [8]:
## LSTM autoencoder
retrunSeq=True
input_data = Input((None,INPUT_SIZE))
encoder=GRU(units=OUTPUT_SIZE*2,  activation='tanh',return_sequences=retrunSeq, name="gru1")(input_data)
encoder=GRU(units=OUTPUT_SIZE,  activation='tanh',return_sequences=retrunSeq, name="gru2")(encoder)
encoder_out=Dense(OUTPUT_SIZE,activation='tanh')(encoder)
encoder_model = Model(inputs=input_data, outputs=encoder_out)

decoder=GRU(units=OUTPUT_SIZE, activation='tanh',return_sequences=retrunSeq,name="de_gru1")(encoder_out)
decoder=GRU(units=OUTPUT_SIZE*2,  activation='tanh', return_sequences=retrunSeq, name="de_gru2")(decoder)
decoder_out=Dense(INPUT_SIZE,activation='relu')(decoder)
autoencoder=Model(input_data,decoder_out)
 
#Compile
#autoencoder = multi_gpu_model(autoencoder, 1)
autoencoder.compile(loss='mean_squared_error',optimizer='adadelta',metrics=['mse'])
autoencoder.summary()  #Print

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', patience = 3,verbose = 1, factor=0.5, min_lr = 0.000001)
#Train
autoencoder.fit(x_train_mid, x_train_mid, batch_size=batch_size, epochs=epochs, validation_data=(x_val_mid, x_val_mid), 
          shuffle=True, verbose=1,callbacks=[learning_rate_reduction])
#Predict
encoded_latent_train = encoder_model.predict(x_train_mid)
encoded_latent_test = encoder_model.predict(x_test_mid)
encoded_latent_val = encoder_model.predict(x_val_mid)
if retrunSeq:
    encoded_latent_train=encoded_latent_train.reshape(-1,OUTPUT_SIZE*TIME_STEPS)
    encoded_latent_test=encoded_latent_test.reshape(-1,OUTPUT_SIZE*TIME_STEPS)
    encoded_latent_val=encoded_latent_val.reshape(-1,OUTPUT_SIZE*TIME_STEPS)

input_representation = Input(shape=(OUTPUT_SIZE*TIME_STEPS,))
latent_vector = Dense(16, activation='relu')(input_representation)
preddiction = Dense(1, activation='relu')(latent_vector)
model2 = Model(input= input_representation,output=preddiction)
#Compile
#model2 = multi_gpu_model(model2, 2)
model2.compile(loss='mean_squared_error',optimizer='adadelta',metrics=['mse'])
model2.summary()  #Print

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', patience = 3,verbose = 1, factor=0.5, min_lr = 0.00001)
#Train
f = open('result_epoch_rnn_ae.csv','w')
for epoch in range(epochs):
    model2.fit(encoded_latent_train, y_train, batch_size=batch_size, nb_epoch=1, validation_data=(encoded_latent_val, y_val), 
              shuffle=True, verbose=1,callbacks=[learning_rate_reduction])
    pre_test_score = model2.evaluate(encoded_latent_test, y_test, verbose=0)
    pre_val_score  = model2.evaluate(encoded_latent_val, y_val, verbose=0)
    f.write(str(epoch)+',Pre Test score,'+ str(pre_test_score[0])+',Pre Val_score,'+str(pre_val_score[0])+'\n')
    print('Pre Test score:', pre_test_score[0])
f.close()


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, 10)          0         
_________________________________________________________________
gru1 (GRU)                   (None, None, 40)          6120      
_________________________________________________________________
gru2 (GRU)                   (None, None, 20)          3660      
_________________________________________________________________
dense_5 (Dense)              (None, None, 20)          420       
_________________________________________________________________
de_gru1 (GRU)                (None, None, 20)          2460      
_________________________________________________________________
de_gru2 (GRU)                (None, None, 40)          7320      
_________________________________________________________________
dense_6 (Dense)              (None, None, 10)          410 

D:\anaconda\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_mse,loss,mse,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


29529/29529 [==============================] - 29s 988us/step - loss: 0.6982 - mse: 0.6982 - val_loss: 0.6975 - val_mse: 0.6975
Epoch 3/20
29529/29529 [==============================] - 27s 921us/step - loss: 0.6982 - mse: 0.6982 - val_loss: 0.6974 - val_mse: 0.6974
Epoch 4/20
29529/29529 [==============================] - 29s 968us/step - loss: 0.6982 - mse: 0.6982 - val_loss: 0.6974 - val_mse: 0.6974
Epoch 5/20
29529/29529 [==============================] - 27s 898us/step - loss: 0.6981 - mse: 0.6981 - val_loss: 0.6974 - val_mse: 0.6974
Epoch 6/20
29529/29529 [==============================] - 26s 880us/step - loss: 0.6981 - mse: 0.6981 - val_loss: 0.6974 - val_mse: 0.6974
Epoch 7/20
29529/29529 [==============================] - 26s 894us/step - loss: 0.6981 - mse: 0.6981 - val_loss: 0.6974 - val_mse: 0.6974
Epoch 8/20
29529/29529 [==============================] - 29s 993us/step - loss: 0.6981 - mse: 0.6981 - val_loss: 0.6974 - val_mse: 0.6974
Epoch 9/20
29529/29529 [==============

D:\anaconda\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
D:\anaconda\lib\site-packages\ipykernel_launcher.py:46: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 120)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 16)                1936      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 17        
Total params: 1,953
Trainable params: 1,953
Non-trainable params: 0
_________________________________________________________________
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 2s 74us/step - loss: 3.6381 - mse: 3.6381 - val_loss: 2.9961 - val_mse: 2.9961
Pre Test score: 2.9513669614639633
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 2s 57us/step - loss: 2.6590 - mse: 2.6590 - val_loss: 2.6720 

In [9]:
## LSTM SelfAttention

model = Sequential()
model.add(keras.layers.Bidirectional(keras.layers.GRU(units=OUTPUT_SIZE, input_shape=input_shape,return_sequences=True)))
model.add(keras.layers.Bidirectional(keras.layers.GRU(units=OUTPUT_SIZE,return_sequences=True)))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
#model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(Flatten()) #Pull into one-dimensional data
model.add(Dense(16, activation='relu')) 
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='relu')) #Fully connected layer
# model.add(Dense(1, activation='relu')) #Fully connected layer
 
#Compile
#model = multi_gpu_model(model, 1)
model.compile(loss='mean_squared_error',optimizer='adadelta',metrics=['mse'])
#model.summary()  #Print

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', patience = 3,verbose = 1, factor=0.5, min_lr = 0.0000004)
#Train
f = open('result_epoch_rnn_SA.csv','w')
for epoch in range(epochs):
    model.fit(x_train_mid, y_train, batch_size=batch_size, nb_epoch=1, validation_data=(x_val_mid, y_val), 
              shuffle=True, verbose=1,callbacks=[learning_rate_reduction])
    pre_test_score = model.evaluate(x_test_mid, y_test, verbose=0)
    pre_val_score  = model.evaluate(x_val_mid, y_val, verbose=0)
    f.write(str(epoch)+',Pre Test score,'+ str(pre_test_score[0])+',Pre Val_score,'+str(pre_val_score[0])+'\n')
    print('Pre Test score:', pre_test_score[0])
f.close()

D:\anaconda\lib\site-packages\ipykernel_launcher.py:24: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 29s 983us/step - loss: 2.9125 - mse: 2.9125 - val_loss: 2.4290 - val_mse: 2.4290


D:\anaconda\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_mse,loss,mse,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Pre Test score: 2.295050116121929
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
  160/29529 [..............................] - ETA: 27s - loss: 3.2097 - mse: 3.2097

D:\anaconda\lib\site-packages\ipykernel_launcher.py:24: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


29529/29529 [==============================] - 28s 955us/step - loss: 2.2609 - mse: 2.2609 - val_loss: 2.3232 - val_mse: 2.3232
Pre Test score: 2.2115785257876985
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 25s 843us/step - loss: 2.2132 - mse: 2.2132 - val_loss: 2.3648 - val_mse: 2.3648
Pre Test score: 2.285798688550611
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 24s 822us/step - loss: 2.1775 - mse: 2.1775 - val_loss: 2.3355 - val_mse: 2.3355
Pre Test score: 2.2081324314833704
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 24s 819us/step - loss: 2.1599 - mse: 2.1599 - val_loss: 2.2539 - val_mse: 2.2539
Pre Test score: 2.133282559770006
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 25s 831us/step - loss: 2.1406 - mse: 2.1406 - val_loss: 2.3020 - va

In [10]:
## LSTM Bi
model = Sequential()
#model.add(keras.layers.Embedding(input_dim=INPUT_SIZE, output_dim=16,mask_zero=True))
model.add(keras.layers.Bidirectional(keras.layers.GRU(units=OUTPUT_SIZE, input_shape=input_shape,return_sequences=True)))
model.add(keras.layers.Bidirectional(keras.layers.GRU(units=OUTPUT_SIZE,return_sequences=True)))

model.add(Flatten()) #Pull into one-dimensional data
model.add(Dense(16, activation='relu')) 
model.add(Dense(1, activation='relu')) #Fully connected layer
# model.add(Dense(1, activation='relu')) #Fully connected layer
 
#Compile
#model = multi_gpu_model(model, 1)
model.compile(loss='mean_squared_error',optimizer='adadelta',metrics=['mse'])
#model.summary()  #Print

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', patience = 3,verbose = 1, factor=0.5, min_lr = 0.00000001)
#Train
f = open(datasetName+'result_epoch_rnn_bi.csv','w')
for epoch in range(epochs):
    model.fit(x_train_mid, y_train, batch_size=batch_size, nb_epoch=1, validation_data=(x_val_mid, y_val), 
              shuffle=True, verbose=1,callbacks=[learning_rate_reduction])
    pre_test_score = model.evaluate(x_test_mid, y_test, verbose=0)
    pre_val_score  = model.evaluate(x_val_mid, y_val, verbose=0)
    f.write(str(epoch)+',Pre Test score,'+ str(pre_test_score[0])+',Pre Val_score,'+str(pre_val_score[0])+'\n')
    print('Pre Test score:', pre_test_score[0])
f.close()

D:\anaconda\lib\site-packages\ipykernel_launcher.py:22: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 24s 800us/step - loss: 2.7622 - mse: 2.7622 - val_loss: 2.4184 - val_mse: 2.4184


D:\anaconda\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_mse,loss,mse,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Pre Test score: 2.30385582726281
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
  192/29529 [..............................] - ETA: 20s - loss: 2.7906 - mse: 2.79

D:\anaconda\lib\site-packages\ipykernel_launcher.py:22: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


29529/29529 [==============================] - 22s 734us/step - loss: 2.2525 - mse: 2.2525 - val_loss: 2.5514 - val_mse: 2.5514
Pre Test score: 2.464182255372478
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 22s 760us/step - loss: 2.2077 - mse: 2.2077 - val_loss: 2.3562 - val_mse: 2.3562
Pre Test score: 2.236393084774014
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 21s 725us/step - loss: 2.1792 - mse: 2.1792 - val_loss: 2.3287 - val_mse: 2.3287
Pre Test score: 2.181010166653265
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 22s 756us/step - loss: 2.1601 - mse: 2.1601 - val_loss: 2.3491 - val_mse: 2.3491
Pre Test score: 2.257734971719461
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 22s 753us/step - loss: 2.1472 - mse: 2.1472 - val_loss: 2.4286 - val_

In [11]:
## mlp
model = Sequential()
model.add(Flatten()) #Pull into one-dimensional data
model.add(Dense(32, activation='relu')) 
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='relu')) #Fully connected layer
# model.add(Dense(1, activation='relu')) #Fully connected layer
 
#Compile
#model = multi_gpu_model(model, 1)
model.compile(loss='mean_squared_error',optimizer='adadelta',metrics=['mse'])
#model.summary()  #Print

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', patience = 3,verbose = 1, factor=0.5, min_lr = 0.00000001)
#Train
f = open(datasetName+'result_epoch_mlp.csv','w')
for epoch in range(epochs):
    model.fit(x_train_mid, y_train, batch_size=batch_size, nb_epoch=1, validation_data=(x_val_mid, y_val), 
              shuffle=True, verbose=1,callbacks=[learning_rate_reduction])
    pre_test_score = model.evaluate(x_test_mid, y_test, verbose=0)
    pre_val_score  = model.evaluate(x_val_mid, y_val, verbose=0)
    f.write(str(epoch)+',Pre Test score,'+ str(pre_test_score[0])+',Pre Val_score,'+str(pre_val_score[0])+'\n')
    print('Pre Test score:', pre_test_score[0])
f.close()
model.summary() 

D:\anaconda\lib\site-packages\ipykernel_launcher.py:19: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 2s 82us/step - loss: 5.1188 - mse: 5.1188 - val_loss: 2.6868 - val_mse: 2.6868
Pre Test score: 2.5223091138149676
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 2s 63us/step - loss: 2.2804 - mse: 2.2804 - val_loss: 2.3482 - val_mse: 2.3482
Pre Test score: 2.243026273933014
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 2s 66us/step - loss: 2.2281 - mse: 2.2281 - val_loss: 2.3577 - val_mse: 2.3577
Pre Test score: 2.2294891702444875
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 2s 67us/step - loss: 2.2069 - mse: 2.2069 - val_loss: 2.3390 - val_mse: 2.3390
Pre Test score: 2.208124977087398
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 2s 67us/step - l